## DataFrame

1. 테스트데이터를 만들때 `createDataFrame(data,schema)` 많이 사용

2. 명시적 스키마 지정 가능 `StructType`

- StructType : 행 전체의 구조를 정의하는 틀 , 안에는 반드시 리스트가 들어간다
- StructField(name,dataType,nullable)
    - nullable: Null허용 여부
 
3. 외부파일 불러올때 read

4. 눈으로 확인 : show, printSchema

In [11]:
from pyspark.sql import (
    Row,
    SparkSession)
from pyspark.sql.functions import col, asc, desc
from pyspark.sql.types import *

In [4]:
spark=(
    SparkSession
    .builder
    .appName("spark-DataFrame")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

In [40]:
data=[
    ("James",30,"M"),
    ("Anna",25,"F"),
    ("Stark",40,"M")
]
columns=["Name","Age","Gender"]

df=spark.createDataFrame(data,schema=columns).cache()
df.show()

+-----+---+------+
| Name|Age|Gender|
+-----+---+------+
|James| 30|     M|
| Anna| 25|     F|
|Stark| 40|     M|
+-----+---+------+



In [42]:
data = [
    ("James", 30, 85.5, True),
    ("Anna", 25, 90.2, False),
    (None, 40, None, None) 
]

schema = StructType([
    StructField("Name", StringType(), True),    
    StructField("Age", IntegerType(), True),    
    StructField("Score", DoubleType(), True),   
    StructField("Is_Pass", BooleanType(), True) 
])
df = spark.createDataFrame(data, schema=schema).cache()
df.show()

+-----+---+-----+-------+
| Name|Age|Score|Is_Pass|
+-----+---+-----+-------+
|James| 30| 85.5|   true|
| Anna| 25| 90.2|  false|
| NULL| 40| NULL|   NULL|
+-----+---+-----+-------+



In [43]:
## CSV 읽을 때 필수 옵션

# header : 첫줄이 제목인지, 데이터인지 확인 False면 자동생성
# -> False :1) .toDF()로 해결 2) withColumRenamed 3) 스키마지정 StructType

# inferSchema : 데이터를 보고 타입을 맞출건지 아닌지 
# False: 모든 데이터를 문자열로 읽음 , True: 스파크가 데이터를 보고 타입 자동 맞춤

In [44]:
df_csv=spark.read.csv("file:///workspace/data/house_price.csv",
                      header=False,
                      inferSchema=True)
df_csv.show()

+-------+-----+---+
|    _c0|  _c1|_c2|
+-------+-----+---+
|  seoul|10000|  3|
|  seoul|10000|  5|
|  seoul|40000|  7|
|  busan| 5000|  7|
|incheon| 4000|  2|
|  busan| 9000|  4|
|incheon| 5000|  7|
|  seoul| 4000|  2|
|  busan| 8000|  9|
+-------+-----+---+



In [45]:
df_csv=spark.read.csv("file:///workspace/data/house_price.csv",
                      header=False,
                      inferSchema=True).toDF("City","Price","Count")
df_csv.show()

+-------+-----+-----+
|   City|Price|Count|
+-------+-----+-----+
|  seoul|10000|    3|
|  seoul|10000|    5|
|  seoul|40000|    7|
|  busan| 5000|    7|
|incheon| 4000|    2|
|  busan| 9000|    4|
|incheon| 5000|    7|
|  seoul| 4000|    2|
|  busan| 8000|    9|
+-------+-----+-----+



In [46]:
# withColumnRenamed 
# 데이터를 이미 읽어버린 상태에서 특정 컬럼 만 바꾸고 싶을때 

In [47]:
df_csv=spark.read.csv("file:///workspace/data/house_price.csv",
                      header=False,
                      inferSchema=True)
df_csv.show()

+-------+-----+---+
|    _c0|  _c1|_c2|
+-------+-----+---+
|  seoul|10000|  3|
|  seoul|10000|  5|
|  seoul|40000|  7|
|  busan| 5000|  7|
|incheon| 4000|  2|
|  busan| 9000|  4|
|incheon| 5000|  7|
|  seoul| 4000|  2|
|  busan| 8000|  9|
+-------+-----+---+



In [48]:
df_csv=df_csv.withColumnRenamed("_c0","City")
df_csv.show()

+-------+-----+---+
|   City|  _c1|_c2|
+-------+-----+---+
|  seoul|10000|  3|
|  seoul|10000|  5|
|  seoul|40000|  7|
|  busan| 5000|  7|
|incheon| 4000|  2|
|  busan| 9000|  4|
|incheon| 5000|  7|
|  seoul| 4000|  2|
|  busan| 8000|  9|
+-------+-----+---+



In [49]:
# show() 
#20줄을 보여줌 기본값
# 내용이 길어도 자르지 말고 다 보고 싶을때 truncate=False
# 상위 5줄 show(5)

In [50]:
df=spark.read.csv("file:///workspace/data/anime.csv",
                 header=True,
                 inferSchema=True)
# df.show()

In [29]:
# df.show(3)

In [31]:
# 너무 길어서 주석
# df.show(truncate=False)

In [32]:
# printSchema() : 뼈대 확인
# 컬럼이름과 데이터 타입, Null허용 여부를 트리구조로 보여줍니다.

In [34]:
df.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- type: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- members: integer (nullable = true)



In [52]:
# describe() 기초통계 확인 
# 숫자 데이터의 개수(count), 평균(mean), 표준편차(stddev), 최소(min), 최대(max)를 계산합니다.
# 원하는 컬럼만 보고 싶을때는 따옴표로 감싸서 문자열로 넣어줘야함

In [53]:
df.describe("members","rating").show()

+-------+-----------------+-----------------+
|summary|          members|           rating|
+-------+-----------------+-----------------+
|  count|            12294|            12064|
|   mean|18071.33886448674|6.473901690981445|
| stddev|54820.67692490701|1.026746306898068|
|    min|                5|             1.67|
|    max|          1013917|             10.0|
+-------+-----------------+-----------------+

